In [60]:
!pip install pyAudioAnalysis

In [61]:
!pip install eyed3
!pip install pydub

In [62]:
!pip install ffmpeg

In [63]:
from glob import glob
import IPython.display as ipd
import matplotlib.pyplot as plt
from pyAudioAnalysis import audioBasicIO as aIo
from pyAudioAnalysis import ShortTermFeatures as sf
import librosa
import librosa.display
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [64]:
file_paths = glob('../Raw/Video/Segmented/*')

In [65]:
data = pd.read_csv('OpinionLevelSentiment.csv', names = ['Seg Start', 'Segment end', 'Name','Seg No', 'Review'])

In [66]:
dataframe = pd.DataFrame(data)
dataframe

,Seg Start,Segment end,Name,Seg No,Review
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0
...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8


In [67]:
dataframe['Polarity'] = dataframe['Review']
for i in range(len(dataframe)):
    if(dataframe['Review'][i]>0):
        #positive
        dataframe['Polarity'][i] = 1
    else:
        #negative
        dataframe['Polarity'][i] = 0

In [68]:
dataframe

,Seg Start,Segment end,Name,Seg No,Review,Polarity
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4,1.0
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0,1.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2,0.0
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0,1.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0,1.0
...,...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4,1.0
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4,1.0
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4,1.0
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8,1.0


In [69]:
p = 0
n = 0
for i in range(2199):
    if(dataframe['Polarity'][i] == 1):
        p+=1
    else:
        n+=1

In [323]:
p

array([[1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [71]:
n

1119

In [72]:
def load_file(filepath):
    y,sr = librosa.load(filepath)
    return (y,sr)

In [73]:
def divide_segments(y_time,start):
    # divide into 50 ms segments
    # 50ms -> means 1102 samples    
    y_segmented = y_time[start:start+1102]
    return y_segmented

In [74]:
def stft(y_seg , sr, window, step):
    [f,f_name] = sf.feature_extraction(y_seg,sr, window, step,  deltas = False)
    return f

In [75]:
def feature_matricization(y_time):
    y_seg = divide_segments(y_time,0)
    y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
    y_stft = y_stft.reshape(34)
    
    start = 1111
    y_stfts = y_stft
    while(start <= len(y_time)-1111):
        y_seg = divide_segments(y_time,start)
        #print(y_seg)
        start += 1111
        y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
        #y_stft_arr = np.asarray(y_stft).T
        y_stft = y_stft.reshape(34)
        y_stfts = np.vstack((y_stfts, y_stft))
        #print(y_stft_arr.shape)

        #np.column_stack((y_stfts, y_stft_arr))
        
    if(len(y_stfts)<300):
        #cal how less
        x = 300 - len(y_stfts)%300
        zeros_arr = np.zeros(shape = (x,34))
        x_arr = np.asanyarray(y_stfts)
        x_arr = x_arr.reshape(-1,34)
        x_final = np.vstack((x_arr,zeros_arr))
        
        return x_final
    if(len(y_stfts)>=300):
        y_stfts = y_stfts[0:300]
        return np.asanyarray(y_stfts)

In [138]:
file_path = 'C:/Users/Tanvi Kandalla/Documents/GAIP/NUS-Project-Audio-Sentiment-Analysis-/Raw/Video/Segmented/' + dataframe['Name'][0] +  '_' + dataframe['Seg No'][0].__str__() +'.mp4'
label = dataframe['Polarity'][0]
labels = np.asarray(label)
y,sr = librosa.load(file_path)
y_time = librosa.frames_to_time(y)
feat = feature_matricization(y_time)
features = feat
print(len(file_path))
for i in range(len(file_path)):
    if(i!=0):
        #print(i)
        file_path = 'C:/Users/Tanvi Kandalla/Documents/GAIP/NUS-Project-Audio-Sentiment-Analysis-/Raw/Video/Segmented/' + dataframe['Name'][i] +  '_' + dataframe['Seg No'][i].__str__() +'.mp4'
        label = dataframe['Polarity'][i]
        labels = np.vstack((labels,label))
        y,sr = librosa.load(file_path)
        y_time = librosa.frames_to_time(y)
        feat = feature_matricization(y_time)
        features = np.vstack((features,feat))

114


In [139]:
labels

array([[1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],

In [140]:
features.shape

(34200, 34)

In [141]:
# divide features into 300 * 34 segements

In [142]:
# no of segments
indices = features.shape[0]/300
indices

114.0

In [143]:
features_segments = np.split(features,indices)

In [144]:
feat_seg_arr = np.asanyarray(features_segments)

In [145]:
feat_seg_arr.shape

(114, 300, 34)

In [146]:
labels = labels.astype('float32')

In [147]:
feat_seg_arr

array([[[8.35603996e-02, 1.28462087e-01, 3.06783807e+00, ...,
         2.19786045e-02, 2.39929811e-03, 1.88485295e-02],
        [7.17529519e-02, 4.10557022e-02, 3.16602963e+00, ...,
         3.15931190e-02, 2.99395294e-03, 1.44249346e-02],
        [8.26521344e-02, 8.54380210e-02, 2.66964478e+00, ...,
         6.96941900e-03, 4.96499094e-04, 9.12360480e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.16257947e-01, 8.10136813e-02, 3.18858525e+00, ...,
         6.24177729e-03, 1.22623693e-03, 1.15392452e-02],
        [8.81017257e-02, 6.77711849e-02, 3.16352363e+00, ...,
         1.71480622e-02, 2.68091891e-03, 1.19572139e-02],
        [1.32606721e-01, 

In [148]:
feat_seg_arr[0][1]

array([ 7.17529519e-02,  4.10557022e-02,  3.16602963e+00,  3.50793637e-01,
        2.92696147e-01,  1.88276576e+00,  0.00000000e+00,  5.29945554e-01,
       -2.16972425e+01,  1.43763990e+00,  3.59081111e-01,  4.89837881e-01,
        3.00855217e-01, -1.11760307e-01, -2.12184171e-01, -1.26670672e-01,
        2.45389000e-01,  9.70260114e-02, -2.08469756e-01, -3.77326015e-01,
       -5.04313857e-02,  3.40617380e-02,  4.19158078e-03,  2.54604638e-02,
        4.70100344e-02,  1.34886251e-02,  3.81588418e-03,  1.12019863e-02,
        1.07353837e-03,  3.41334481e-03,  1.94003880e-02,  3.15931190e-02,
        2.99395294e-03,  1.44249346e-02])

In [149]:
feat_seg_arr_norm = feat_seg_arr
for i in range(feat_seg_arr.shape[0]):
    for j in range(feat_seg_arr.shape[1]):
        for k in range(feat_seg_arr.shape[2]):
            if(feat_seg_arr[i][j].max()!=0):
                
                feat_seg_arr_norm[i][j][k] = feat_seg_arr[i][j][k]/feat_seg_arr.max()

KeyboardInterrupt: 

In [ ]:
feat_seg_arr_norm

In [150]:
features.shape

(34200, 34)

In [151]:
print(labels)

[[1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [152]:
feat_seg_arr.shape

(114, 300, 34)

In [153]:
labels.shape

(114, 1)

In [154]:
# train test split

In [398]:
from sklearn.model_selection import train_test_split

In [399]:
X_train, X_test, y_train, y_test = train_test_split(feat_seg_arr, labels,random_state = 10)

In [400]:
print(X_train[10] == X_train[1])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [401]:
X_train.shape

(85, 300, 34)

In [402]:
y_train.shape

(85, 1)

In [403]:
import tensorflow as tf

In [404]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import TimeDistributed,LSTM,BatchNormalization

In [405]:
"""model = Sequential()"""

'model = Sequential()'

In [406]:
"""model = Sequential()
model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(300,34)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))

model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(300,34)))
model.add(layers.Activation('relu'))
#model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))

model.add(layers.Dropout(0.1))
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.Dense(128))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))"""

"model = Sequential()\nmodel.add(layers.Conv1D(256, 5,padding='same',\n                 input_shape=(300,34)))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Dropout(0.2))\nmodel.add(layers.Conv1D(128, 5,padding='same'))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.MaxPooling1D(pool_size=(4)))\n\nmodel.add(layers.Conv1D(256, 5,padding='same',\n                 input_shape=(300,34)))\nmodel.add(layers.Activation('relu'))\n#model.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Dropout(0.2))\nmodel.add(layers.Conv1D(128, 5,padding='same'))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.MaxPooling1D(pool_size=(4)))\n\nmodel.add(layers.Dropout(0.1))\nmodel.add(layers.Flatten())\nmodel.add(layers.Dense(256))\nmodel.add(layers.Dense(128))\nmodel.add(layers.Dense(1))\nmodel.add(layers.Activation('sigmoid'))"

In [407]:
"""model.compile(optimizer = 'Adam' , loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])"""

"model.compile(optimizer = 'Adam' , loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])"

In [420]:
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(layers.Conv1D(96,5, input_shape=(300,34), padding='same'))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
AlexNet.add(layers.MaxPooling1D(pool_size=8, padding='same'))

#2nd Convolutional Layer
AlexNet.add(layers.Conv1D(256,5, padding='same'))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
AlexNet.add(layers.MaxPooling1D(pool_size=4, padding='same'))

#3rd Convolutional Layer
AlexNet.add(layers.Conv1D(384,5, padding='same'))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))

#4th Convolutional Layer
AlexNet.add(layers.Conv1D(384,5, padding='same'))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
#5th Convolutional Layer
AlexNet.add(layers.Conv1D(384,5, padding='same'))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
AlexNet.add(layers.MaxPooling1D(pool_size=4, padding='same'))
AlexNet.add(layers.Dropout(0.1))
#Passing it to a Fully Connected layer
AlexNet.add(layers.Flatten())
# 1st Fully Connected Layer
AlexNet.add(layers.Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(layers.Dropout(0.2))

#2nd Fully Connected Layer
AlexNet.add(layers.Dense(4096))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('relu'))
#Add Dropout
AlexNet.add(layers.Dropout(0.4))

#Output Layer
AlexNet.add(layers.Dense(1))
AlexNet.add(layers.BatchNormalization())
AlexNet.add(layers.Activation('sigmoid'))

#Model Summary
AlexNet.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_96 (Conv1D)          (None, 300, 96)           16416     
                                                                 
 batch_normalization_59 (Bat  (None, 300, 96)          384       
 chNormalization)                                                
                                                                 
 activation_127 (Activation)  (None, 300, 96)          0         
                                                                 
 max_pooling1d_64 (MaxPoolin  (None, 38, 96)           0         
 g1D)                                                            
                                                                 
 conv1d_97 (Conv1D)          (None, 38, 256)           123136    
                                                                 
 batch_normalization_60 (Bat  (None, 38, 256)        

In [421]:
AlexNet.compile(optimizer = 'Adam' , loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [422]:
AlexNet.fit(X_train,y_train,epochs = 25)

Epoch 1/25
3/3 [==============================] - 7s 308ms/step - loss: 0.7962 - accuracy: 0.5647
Epoch 2/25
3/3 [==============================] - 1s 310ms/step - loss: 0.6648 - accuracy: 0.6000
Epoch 3/25
3/3 [==============================] - 1s 281ms/step - loss: 0.5495 - accuracy: 0.7294
Epoch 4/25
3/3 [==============================] - 1s 288ms/step - loss: 0.5644 - accuracy: 0.7412
Epoch 5/25
3/3 [==============================] - 1s 302ms/step - loss: 0.5423 - accuracy: 0.7412
Epoch 6/25
3/3 [==============================] - 1s 284ms/step - loss: 0.5451 - accuracy: 0.7765
Epoch 7/25
3/3 [==============================] - 1s 274ms/step - loss: 0.5085 - accuracy: 0.7765
Epoch 8/25
3/3 [==============================] - 1s 287ms/step - loss: 0.5461 - accuracy: 0.7294
Epoch 9/25
3/3 [==============================] - 1s 282ms/step - loss: 0.5033 - accuracy: 0.7765
Epoch 10/25
3/3 [==============================] - 1s 276ms/step - loss: 0.5114 - accuracy: 0.7647
Epoch 11/25
3/3 [==

In [423]:
p = AlexNet.predict(X_test)
for i in range(len(p)):
    if p[i]>0.9:
        p[i] = 1.0
    else:
        p[i] = 0.0

1/1 [==============================] - 1s 902ms/step


In [424]:
from sklearn.metrics import accuracy_score

In [425]:
accuracy = accuracy_score(y_test,p)
accuracy

0.5862068965517241